# 🧭 Vision globale de la base de données Olist

La base de données Olist modélise l’ensemble du processus e-commerce, **de la commande jusqu’à la satisfaction client**, en passant par les produits, les vendeurs et les paiements.  


---

## 🗃️Dictionnaire des tables

### **olist_orders_dataset**
**Clé primaire :** `order_id`

| Champ | Description |
|-------|-------------|
| order_id | Identifiant unique de la commande |
| customer_id | Identifiant du client (clé étrangère vers `olist_customers_dataset`) |
| order_status | Statut de la commande (delivered, shipped, canceled, etc.) |
| order_purchase_timestamp | Date et heure d’achat |
| order_approved_at | Date d’approbation du paiement |
| order_delivered_carrier_date | Date d’expédition au transporteur |
| order_delivered_customer_date | Date de livraison au client |
| order_estimated_delivery_date | Date estimée de livraison |

---

### **olist_customers_dataset**
**Clé primaire :** `customer_id`

| Champ | Description |
|-------|-------------|
| customer_id | Identifiant unique du client |
| customer_unique_id | Identifiant client unique (utile pour détecter les clients récurrents) |
| customer_zip_code_prefix | Code postal du client |
| customer_city | Ville du client |
| customer_state | État/région du client |

---

### **olist_order_items_dataset**
**Clé primaire :** `(order_id, order_item_id)`

| Champ | Description |
|-------|-------------|
| order_id | Identifiant de la commande |
| order_item_id | Numéro de l’article dans la commande |
| product_id | Identifiant du produit |
| seller_id | Identifiant du vendeur |
| shipping_limit_date | Date limite d’expédition |
| price | Prix du produit |
| freight_value | Coût du transport |

---

### **olist_order_payments_dataset**
**Clé primaire :** `(order_id, payment_sequential)`

| Champ | Description |
|-------|-------------|
| order_id | Identifiant de la commande |
| payment_sequential | Numéro du paiement (utile si plusieurs paiements pour une même commande) |
| payment_type | Type de paiement (credit_card, boleto, voucher, etc.) |
| payment_installments | Nombre de mensualités |
| payment_value | Montant du paiement |

---

### **olist_order_reviews_dataset**
**Clé primaire :** `review_id`

| Champ | Description |
|-------|-------------|
| review_id | Identifiant unique de l’avis |
| order_id | Identifiant de la commande |
| review_score | Note donnée (1 à 5) |
| review_comment_title | Titre du commentaire |
| review_comment_message | Message du commentaire |
| review_creation_date | Date de création de l’avis |
| review_answer_timestamp | Date de réponse de l’équipe Olist |

---

### **olist_products_dataset**
**Clé primaire :** `product_id`

| Champ | Description |
|-------|-------------|
| product_id | Identifiant unique du produit |
| product_category_name | Nom de la catégorie du produit (en portugais) |
| product_name_length | Longueur du nom du produit |
| product_description_length | Longueur de la description |
| product_photos_qty | Nombre de photos |
| product_weight_g | Poids en grammes |
| product_length_cm | Longueur en cm |
| product_height_cm | Hauteur en cm |
| product_width_cm | Largeur en cm |

---

### **product_category_name_translation**
**Clé primaire :** `product_category_name`

| Champ | Description |
|-------|-------------|
| product_category_name | Nom de catégorie (en portugais) |
| product_category_name_english | Traduction anglaise de la catégorie |

---

### **olist_sellers_dataset**
**Clé primaire :** `seller_id`

| Champ | Description |
|-------|-------------|
| seller_id | Identifiant du vendeur |
| seller_zip_code_prefix | Code postal du vendeur |
| seller_city | Ville du vendeur |
| seller_state | État/région du vendeur |

---

### **olist_geolocation_dataset**
**Clé primaire :** *(aucune)*

| Champ | Description |
|-------|-------------|
| geolocation_zip_code_prefix | Code postal |
| geolocation_lat | Latitude |
| geolocation_lng | Longitude |
| geolocation_city | Ville |
| geolocation_state | État/région |

*(Cette table peut être utilisée pour enrichir `customers` et `sellers` avec des coordonnées GPS.)*

---

### **dcfb4959-2b00-4550-95d9-b274524f760a.csv**
👉 À identifier — le nom ne permet pas de déterminer son rôle.  
Une inspection du contenu permettra de l’ajouter proprement au dictionnaire.

---

## 🔗 3. Relations principales entre les tables

| Table Source | Clé étrangère | Table Cible | Type de relation |
|---------------|---------------|--------------|------------------|
| `olist_orders_dataset` | `customer_id` | `olist_customers_dataset` | Plusieurs commandes pour un client |
| `olist_order_items_dataset` | `order_id` | `olist_orders_dataset` | Une commande contient plusieurs articles |
| `olist_order_items_dataset` | `product_id` | `olist_products_dataset` | Un article correspond à un produit |
| `olist_order_items_dataset` | `seller_id` | `olist_sellers_dataset` | Un article est vendu par un vendeur |
| `olist_order_reviews_dataset` | `order_id` | `olist_orders_dataset` | Une commande peut avoir un avis |
| `olist_order_payments_dataset` | `order_id` | `olist_orders_dataset` | Une commande peut avoir plusieurs paiements |
| `olist_products_dataset` | `product_category_name` | `product_category_name_translation` | Traduction de catégorie |
| `olist_customers_dataset` / `olist_sellers_dataset` | `*_zip_code_prefix` | `olist_geolocation_dataset` | Enrichissement géographique |

---

## 📊 4. Indicateurs clés et dérivés

### 🧾 Commandes
- Nombre total de commandes : `COUNT(order_id)`
- Délai moyen de livraison : `AVG(order_delivered_customer_date - order_purchase_timestamp)`
- Taux d’annulation : % `order_status = 'canceled'`
- Respect du délai estimé : % livrées avant `order_estimated_delivery_date`

### 👥 Clients
- Nombre de clients uniques : `COUNT(DISTINCT customer_unique_id)`
- Taux de récurrence : clients ≥ 2 commandes / total
- Panier moyen : `SUM(price) / COUNT(DISTINCT order_id)`

### 💰 Paiements
- Revenu total : `SUM(payment_value)`
- Moyenne du nombre de mensualités : `AVG(payment_installments)`
- Répartition par moyen de paiement : `payment_type`

### 📦 Produits
- Nombre de produits vendus : `COUNT(product_id)`
- Prix moyen par catégorie : `AVG(price)`
- Top 10 catégories : `GROUP BY product_category_name_english ORDER BY SUM(price)`

### 🏬 Vendeurs
- Nombre de vendeurs actifs : `COUNT(DISTINCT seller_id)`
- Délai moyen d’expédition : `AVG(order_delivered_carrier_date - order_approved_at)`
- Répartition géographique : `seller_state`

### ⭐ Avis & Satisfaction
- Note moyenne : `AVG(review_score)`
- % avis positifs (≥4) / négatifs (≤2)
- Corrélation note / délai de livraison

### 🗺️ Géographie
- Répartition clients / vendeurs par région
- Distance moyenne client-vendeur (via coordonnées lat/lon)
- Chiffre d’affaires par État

---

## 🧮 5. Indicateurs dérivés globaux

| Thème | Indicateur | Tables impliquées |
|--------|-------------|-------------------|
| Performance | Taux de livraison à temps | `orders`, `order_items` |
| Satisfaction | Note moyenne par vendeur | `reviews`, `order_items`, `sellers` |
| Rentabilité | Revenu total par vendeur | `order_items`, `payments`, `sellers` |
| Client | CLV (Customer Lifetime Value) | `orders`, `payments`, `customers` |
| Produits | Top produits les plus vendus | `order_items`, `products` |
| Paiements | Part des paiements par carte | `payments` |

---

## 🧱 6. Modélisation analytique (modèle en étoile)

L’objectif de la modélisation analytique est d’organiser les données Olist dans un **modèle en étoile (Star Schema)** pour faciliter :
- la **visualisation dans Power BI / Tableau / Looker Studio**,
- la **rapidité des agrégations**,
- la **cohérence des indicateurs** à travers toutes les analyses.

**Table de faits principale :**
- `fact_orders` → (fusion de `orders`, `order_items`, `payments`, `reviews`)

**Tables de dimensions :**
- `dim_customers`
- `dim_products`
- `dim_sellers`
- `dim_geolocation`
- `dim_reviews`
- `dim_date`


→ Structure idéale pour Power BI / Tableau / Looker Studio.


---

### 🧾 Table de faits : `fact_orders`

Cette table fusionne les informations des commandes, des paiements, des avis et des produits vendus.  
Chaque ligne correspond à **un article commandé** (niveau "order item").
**Clé primaire :** `order_id` + `order_item_id`
| Champ | Source | Description |
|-------|---------|-------------|
| `order_id` | orders | Identifiant unique de commande |
| `order_item_id` | order_items | Numéro de ligne de commande |
| `customer_id` | orders | Référence du client |
| `seller_id` | order_items | Référence du vendeur |
| `product_id` | order_items | Référence du produit |
| `purchase_date` | orders | Date d’achat |
| `expedition_date` | orders | Date de dépôt au transporteur |
| `delivery_date` | orders | Date réelle de livraison |
| `estimated_delivery_date` | orders | Date estimée de livraison |
| `price` | order_items | Prix de l’article |
| `freight_value` | order_items | Coût du transport |
| `payment_value` | payments | Montant total payé |
| `payment_type` | payments | Type de paiement |
| `review_score` | reviews | Note donnée par le client |
| `delivery_delay_days` | dérivé | `delivery_date - estimated_delivery_date` |
| `delivery_time_days` | dérivé | `delivery_date - purchase_date` |
| `is_late_delivery` | dérivé | Booléen : en retard ou non |
| `total_revenue` | dérivé | `price + freight_value` |

---

### 🧱 Tables de dimensions

#### 🧍 `dim_customers`
**Clé primaire :** customer_id
| Champ | Description |
|--------|-------------|
| customer_id | Identifiant du client |
| customer_unique_id | Identifiant unique (permet de regrouper plusieurs commandes) |
| customer_city | Ville du client |
| customer_state | État / région |
| customer_zip_code_prefix | Code postal |

→ **Permet d’analyser les ventes et comportements clients par région, récurrence, etc.**

---

#### 🏬 `dim_sellers`
**Clé primaire :** seller_id
| Champ | Description |
|--------|-------------|
| seller_id | Identifiant du vendeur |
| seller_city | Ville du vendeur |
| seller_state | État / région |
| seller_zip_code_prefix | Code postal |

→ **Utilisée pour analyser la performance et la couverture géographique des vendeurs.**

---

#### 📦 `dim_products`
**Clé primaire :** product_id
| Champ | Description |
|--------|-------------|
| product_id | Identifiant du produit |
| product_category_name | Catégorie (portugais) |
| product_category_name_english | Catégorie (anglais, via table de traduction) |
| product_weight_g | Poids en grammes |
| product_length_cm | Longueur |
| product_height_cm | Hauteur |
| product_width_cm | Largeur |
| product_volume_cm3 | colonne ajouté = Longueur*Hauteur*Largeur(cm) |

→ **Permet d’évaluer les performances produit (CA, satisfaction, volume, poids, etc.).**

---

#### 🗓️ `dim_date`
**Clé primaire :** date
| Champ | Description |
|--------|-------------|
| date | Jour unique |
| year | Année |
| quarter | Trimestre |
| month | Mois |
| week | Semaine |
| day | Jour du mois |
| weekday | Jour de la semaine |

→ **Permet l’analyse temporelle : saisonnalité, tendances, délais moyens, etc.**

---

#### 🌍 `dim_geolocation`
**Clé primaire :** zip_code_prefix + geolocation_lat + geolocation_lng
| Champ | Description |
|--------|-------------|
| zip_code_prefix | Code postal |
| geolocation_lat | Latitude |
| geolocation_lng | Longitude |
| geolocation_city | Ville |
| geolocation_state | État/région |

→ **Utilisée pour cartographier les clients et vendeurs, calculer les distances, etc.**

---
### 'dim_reviews'
**Clé primaire :** `review_id`

| Champ | Description |
|-------|-------------|
| review_id | Identifiant unique de l’avis |
| order_id | Identifiant de la commande |
| review_score | Note donnée (1 à 5) |
| result_concatenation_coms | le concat du titre + message du commentaire |
| review_creation_date | Date de création de l’avis |
| review_answer_timestamp | Date de réponse de l’équipe Olist |


→ **Permet d’uniformiser les catégories dans les visualisations.**

---

### 📊 Exemples de mesures dérivées (KPI)

#### 🔹 Performance logistique
| Indicateur | Formule (DAX ou SQL) |
|-------------|----------------------|
| Délai moyen de livraison | `AVG(DATEDIFF(order_purchase_timestamp, order_delivered_customer_date, DAY))` |
| Taux de livraison à temps | `% commandes où delivery_date <= estimated_delivery_date` |
| Coût moyen du transport | `AVG(freight_value)` |

---

#### 🔹 Performance commerciale
| Indicateur | Formule |
|-------------|----------|
| Chiffre d’affaires total | `SUM(total_revenue)` |
| Panier moyen | `SUM(total_revenue) / COUNT(DISTINCT order_id)` |
| Nombre de commandes | `COUNT(order_id)` |
| Top catégories produits | `SUM(total_revenue) GROUP BY product_category_name_english` |

---

#### 🔹 Satisfaction client
| Indicateur | Formule |
|-------------|----------|
| Note moyenne | `AVG(review_score)` |
| % d’avis positifs | `COUNT(review_score >= 4) / COUNT(*)` |
| Corrélation délai / note | `CORR(delivery_delay_days, review_score)` |

---

#### 🔹 Clients
| Indicateur | Formule |
|-------------|----------|
| Nombre de clients actifs | `COUNT(DISTINCT customer_unique_id)` |
| Taux de réachat | clients ≥ 2 commandes / total clients |
| Valeur vie client (CLV) | `SUM(total_revenue) BY customer_unique_id` |

---

### 🧩 7. Extension possible : modèle en flocon (Snowflake Schema)

Si besoin d’un modèle plus détaillé :
- Séparer `dim_products` et `dim_category_translation`
- Ajouter une table `dim_payment_type`
- Ajouter une table `dim_review` (si plusieurs avis par commande)

Cela permet de **réduire la redondance** et de faciliter les **jointures spécifiques**.

---

### ✅ Synthèse : structure cible Power BI

| Type | Nom de la table | Source principale | Niveau de granularité |
|-------|------------------|-------------------|-----------------------|
| **Fait** | `fact_orders` | Orders + Items + Payments + Reviews | 1 ligne = 1 article commandé |
| **Dimension** | `dim_customers` | Customers + Geolocation | Client |
| **Dimension** | `dim_sellers` | Sellers + Geolocation | Vendeur |
| **Dimension** | `dim_products` | Products + Category Translation | Produit |
| **Dimension** | `dim_date` | Générée | Jour |
| **Dimension** | `dim_geolocation` | Geolocation | Localisation |
| **Dimension (optionnelle)** | `dim_payment_type` | Payments | Type de paiement |

---

➡️ **Résultat attendu :**
Un modèle performant pour explorer :
- le **parcours client** (achat → livraison → satisfaction),
- la **performance commerciale** (CA, marges, délais),
- la **performance logistique** (retards, transport),
- et la **rentabilité par produit / vendeur / région**.
